# Анализ факторов экологии на результаты ЕГЭ

## ЕГЭ

https://ivo.garant.ru/#/document/406882488

### Досрочный период

22 марта (пятница) — география, литература;
26 марта (вторник) — русский язык;
29 марта (пятница) — ЕГЭ по математике базового уровня, ЕГЭ по математике профильного уровня;
2 апреля (вторник) — биология, иностранные языки (английский, испанский, китайский, немецкий, французский) (письменная часть), физика;
5 апреля (пятница) — иностранные языки (английский, испанский, китайский, немецкий, французский) (устная часть);
9 апреля (вторник) — информатика, обществознание;
12 апреля (пятница) — история, химия.

### Резервные дни

15 апреля (понедельник) — русский язык;
18 апреля (четверг) — ЕГЭ по математике базового уровня, ЕГЭ по математике профильного уровня;
19 апреля (пятница) — биология, иностранные языки (английский, испанский, китайский, немецкий, французский) (письменная часть), литература, обществознание, физика;
22 апреля (понедельник) — география, иностранные языки (английский, испанский, китайский, немецкий, французский) (устная часть), информатика, история, химия;

Основной период

23 мая (четверг) — география, литература, химия;
28 мая (вторник) — русский язык;
29 мая (среда) — русский язык;
31 мая (пятница) — ЕГЭ по математике базового уровня, ЕГЭ по математике профильного уровня;
1 июня (суббота) — ЕГЭ по математике базового уровня, ЕГЭ по математике профильного уровня;
4 июня (вторник) — обществознание, физика;
7 июня (пятница) — иностранные языки (английский, испанский, китайский, немецкий, французский) (устная часть), информатика;
8 июня (суббота) — иностранные языки (английский, испанский, китайский, немецкий, французский) (устная часть), информатика;
11 июня (вторник) — биология, иностранные языки (английский, испанский, китайский, немецкий, французский) (письменная часть), история.


### Дополнительный период

4 сентября (среда) — русский язык;
9 сентября (понедельник) — ЕГЭ по математике базового уровня.
23 сентября (понедельник) — ЕГЭ по математике базового уровня, русский язык.

ЕГЭ начинают сдавать с марта и заканчивают в сентябре

In [1]:
import os
import re
import json
import gzip
import time
import zipfile
import requests
import numpy as np
import pandas as pd
from datetime import date
from decimal import Decimal
#from tokens import YANDEX_ORG
#from tokens import YANDEX_GEO # Раскоменить для работы с API Яндекс.Организации и получения координат
#from tokens import DATA_MOS_RU # Раскоменить для работы с API data.mos.ru
#from tokens import MAPS_CO
MAPS_CO = 0 # # Закоментить строку если есть ключ
YANDEX_GEO = 0 # Закоментить строку если есть ключ
COUNTER = 0 # глобальный счетчик, можно использовать для подсчета кол-ва запросов по АПИ
LIMIT = 1000 # лимит запросов по АПИ и кол-во записей в одно запросе
CACHE = {} # глобальный кэш, в процессе скрейпинга не обновлять!
SETS = './sets/' # папка с файлами с наборами

In [12]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_cache(filename='cache.gz'): # загрузка кэша
    global CACHE
    if os.path.exists(filename) and len(CACHE)==0:
        with gzip.open(filename, 'rb') as gzip_ref:
            CACHE = json.load(gzip_ref)
        print(f'Загружено {len(CACHE)} записей из {filename} в кэш')


def save_cache(filename='cache.gz'): # сохранение кэша
    global CACHE
    with gzip.open(filename, 'wb') as gzip_file:
        json_data = json.dumps(CACHE, ensure_ascii=False, default=default_serializer)
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(CACHE)} записей в {filename}')
    
    
def load_gzon(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return json.load(gzip_ref)
    else:
        return []
    

def save_gzon(filename):
    data = globals().get(filename)
    if data is not None:
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(json_data)} записей в {filename}.gz')
    else:
        print(f'Переменная {filename} не найдена')
        
     
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        
    
def mos_geojson(dataset):
    all_data = load_gzon(dataset)
    if(len(all_data) > 0):
        return all_data

    step = LIMIT
    url = "https://apidata.mos.ru/v1"
    folder = "/datasets/"
    params = {
        "api_key": DATA_MOS_RU,
        "$skip": 0
    }
    
    count = int(requests.get(url + folder + dataset + "/count", params=params).text)
    while count > params["$skip"]:
        response = requests.get(url + folder + dataset + "/rows", params=params)
        data = json.loads(response.text)
        for d in data:
            all_data.append(d['Cells'])
        params["$skip"] += step
    json_data = json.dumps(all_data, ensure_ascii=False)
    with gzip.open(SETS + dataset + '.gz', 'wb') as gzip_file:
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(all_data)} записей в {dataset}.gz')
        
    return all_data
        

def get_dataset(url, encoding='utf-8', forma='geojson')->pd.DataFrame:
    if(forma == 'geojson'):
        dataset = pd.DataFrame(mos_geojson(url))
    else:
        filename = ''
        filenames = []
        response = requests.get(url) 
        if response.status_code == 200:
            content_disposition = response.headers.get("Content-Disposition")
            if content_disposition:
                filename = re.findall("filename=(.+)", content_disposition)
                if filename:
                    filename = filename[0]
                    filename = re.sub(r'"', '', filename)
                    with open(filename, "wb") as file:
                        file.write(response.content)
                else:
                    print("Не удалось получить имя файла")
            else:
                print("Не удалось получить информацию о загруженном файле")
        else:
            print("Ошибка при загрузке файла")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            filenames = zip_ref.namelist()
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extract(filenames[0])
        if( forma == 'csv'):
            dataset = pd.read_csv(filenames[0], sep=';', encoding=encoding)
            dataset = dataset.drop(index=0)
            dataset = dataset.drop(columns=[col for col in dataset.columns if col.startswith("Unnamed:")])
        elif(forma == 'json'):
            dataset = pd.read_json(filenames[0], encoding=encoding)
        os.remove(filename)
        os.remove(filenames[0])
    print(dataset.info())
    return dataset

# Функция для запроса к API Яндекса
def yandex_org(row):
    global CAHCE
    load_cache()
    if pd.notna(row['geoData']):
        return row['geoData']
    
    key = f"{row['AdmArea']}_{row['District']}_{row['EDU_NAME']}"
    if key in CACHE:
        return CACHE[key]
    
    url = "https://search-maps.yandex.ru/v1/"
    params = {
        "apikey": YANDEX_ORG,
        "lang": "ru_RU",
        "text": f"Москва, {row['AdmArea']}, {row['District']}, {row['EDU_NAME']}"
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'features' in data and len(data['features']) > 0:
            geo_data = data['features'][0]['geometry']
            CACHE[key] = geo_data
            return geo_data
        else:
            CACHE[key] = None
            return None
    except Exception as e:
        print(f"Ошибка при запросе к API: {e}")
        geo_cache[key] = None
        return None

# Школы

ID:2072

## Датасет с результатами ЕГЭ

[Источник](https://data.mos.ru/opendata/2072)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=200291)

Набор данных «Результаты ЕГЭ» позволяет получить информацию об общих результатах государственной итоговой аттестации по программам среднего общего образования (далее – ГИА-11) каждой образовательной организации города Москвы.

В наборе представлены данные:

- об административных округах, районах, полных наименованиях образовательных организаций;

- о периоде, за который сдавался единый государственный экзамен (далее – ЕГЭ);

- о количестве участников, набравших не менее 200 баллов за 3 экзамена;

- о количестве участников, набравших более 160 баллов за 3 экзамена.

Результаты планируется публиковать ежегодно, таким образом можно будет проследить динамику повышения качества образования в Москве.


Современная ситуация в России

Согласно статье 59 Федерального закона от 29.12.2012 г. № 273-ФЗ «Об образовании в Российской Федерации», ГИА-11 завершает освоение основных образовательных программ среднего общего образования и является обязательной.

К ГИА-11 допускаются обучающиеся, не имеющие академической задолженности, в том числе за итоговое сочинение (изложение), и в полном объеме выполнившие учебный план или индивидуальный учебный план (имеющие годовые отметки по всем учебным предметам учебного плана за каждый год обучения по образовательной программе среднего общего образования не ниже удовлетворительных).

К ГИА-11 по учебным предметам, освоение которых завершилось ранее, допускаются обучающиеся X-XI (XII) классов, имеющие годовые отметки не ниже удовлетворительных по всем учебным предметам учебного плана за предпоследний год обучения.

Выпускники текущего года проходят ГИА-11 по двум обязательным учебным предметам (русский язык и математика) и учебным предметам по выбору.


Современная ситуация в Москве

Участники ГИА-11 могут узнать свои результаты по следующей схеме:

<table>
  <tr><td>обучающиеся образовательных организаций</td><td>в своей образовательной организации</td></tr>
  <tr><td>выпускники прошлых лет</td><td>в региональном центре обработки</td></tr>
</table>

На официальном Портале Мэра и Правительства Москвы www.mos.ru в разделе «Услуги и сервисы» можно оформить подписку на получение уведомлений о готовности результатов ГИА-11.

А знаете ли вы, что...

- Для проведения ГИА-11 предусмотрены следующие сроки: досрочный, основной и дополнительный периоды.

- Срок действия результатов ЕГЭ – 4 года, следующих за годом получения таких результатов.

- Результаты ЕГЭ каждого участника заносятся в федеральную информационную систему, бумажных свидетельств о результатах ЕГЭ не предусмотрено.


Материал подготовлен на основе информации data.mos.ru, dogm.mos.ru, mos.ru.

In [3]:
pd.set_option('display.max_colwidth', None)
exams = get_dataset('2072')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5542 entries, 0 to 5541
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                5542 non-null   int64 
 1   EDU_NAME          5542 non-null   object
 2   global_id         5542 non-null   int64 
 3   YEAR              5542 non-null   object
 4   AdmArea           5542 non-null   object
 5   District          5542 non-null   object
 6   PASSES_OVER_220   5542 non-null   int64 
 7   PASSER_UNDER_160  5542 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 346.5+ KB
None


In [4]:
# чистим имена школ от мусора, делаем все буквы маленькими, убираем кавычки, и др. найденные разночтения
exams['EDU_NAME'] = exams['EDU_NAME'].str.lower().str.strip().str.replace('«', '').str.replace('»', '') \
.str.replace('  ', ' ').str.replace('"', ' ').str.replace('общеобразовательн', 'образовательн').str.replace('№ ', '')
#exams['geoData'] = None
exams.head()

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160
0,8236,государственное бюджетное образовательное учреждение города москвы школа 390 имени генерала п.и. батова,797946035,2016-2017,Восточный административный округ,район Богородское,5,19
1,8237,государственное казенное образовательное учреждение города москвы кадетская школа-интернат 5 преображенский кадетский корпус,797949150,2016-2017,Восточный административный округ,район Богородское,14,47
2,8238,государственное казенное образовательное учреждение города москвы специальная (коррекционная) образовательная школа-интернат 31,797949275,2016-2017,Восточный административный округ,район Вешняки,0,1
3,8239,частное учреждение образовательная организация московская интернациональная школа планета,797949425,2016-2017,Восточный административный округ,район Вешняки,0,6
4,8240,автономная некоммерческая организация средняя образовательная школа колибри,797949570,2016-2017,Восточный административный округ,район Восточное Измайлово,1,5


## Датасет с координатами школ

ID:2263

[Источник](https://data.mos.ru/opendata/2263)

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=217731)

Набор данных содержит информацию об образовательных организациях города Москвы: адреса, контактные телефоны, местоположение на карте, адреса интернет-сайтов учреждений, а также информацию о доступности зданий для лиц с ограниченными возможностями.

В соответствии с Федеральным Законом от 29.12.2012 № 273-ФЗ «Об образовании в Российской Федерации» (Закон) изменилась классификация образовательных организаций. Ранее образовательные организации классифицировались по «типам» и «видам» в зависимости от возраста обучающихся, в настоящее время согласно п.2 ст.10 Закона образование подразделяется на общее, профессиональное, дополнительное и профессиональное обучение, обеспечивающие возможность реализации права на образование в течение всей жизни (непрерывное образование).

Современные московские образовательные организации, как правило, реализуют несколько уровней общего образования (п.4 ст.10 Закона) – дошкольное, начальное общее, основное общее, среднее общее. Они также могут предлагать среднее и высшее профессиональное образование (п.5 ст.10 Закона) и четкой привязки образовательных организаций к прежней классификации (Детский сад → Средняя школа → Учреждение среднего профессионального образования → Высшее учебное заведение) уже нет.

У больших школ появились новые возможности - образовательные организации получили ресурс для повышения доступности реализуемых ими образовательных программ для московских семей. В зависимости от предпочтений семей школа предлагает дополнительное образование: организует работу кружков, секций, курсов различной направленности для детей и взрослых в любом из своих зданий.

Именно поэтому местом предоставления той или иной образовательной услуги должны считаться все здания, входящие в состав образовательной организации. Места оказания услуг общего и профессионального образования, а также – место работы конкретного кружка вы можете уточнить у администрации или на официальном сайте образовательной организации.

In [5]:
schools = get_dataset('2263')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 0 to 577
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   OrgType                                578 non-null    object 
 1   ReorganizationStatus                   578 non-null    object 
 2   IDEKIS                                 578 non-null    int64  
 3   FullName                               578 non-null    object 
 4   ShortName                              578 non-null    object 
 5   Number                                 475 non-null    float64
 6   INN                                    578 non-null    object 
 7   KPP                                    578 non-null    object 
 8   OGRN                                   578 non-null    object 
 9   LegalOrganization                      578 non-null    object 
 10  Subordination                          578 non-null    object 
 11  ChiefN

In [6]:
schools['FullName'] = schools['FullName'].str.lower().str.strip().str.replace('«', '').str.replace('»', '') \
.str.replace('  ', ' ').str.replace('"', ' ').str.replace('общеобразовательн', 'образовательн').str.replace('№ ', '')
# Преобразование значения в колонке 'geoData' в список координат
# schools['geoData'] = schools['geoData'].str.replace('{coordinates=[', '').str.replace('], type=Polygon}', '')
schools.head()

,OrgType,ReorganizationStatus,IDEKIS,FullName,ShortName,Number,INN,KPP,OGRN,LegalOrganization,...,Email,WebSite,EducationPrograms,EducationalServices,LicensingAndAccreditation,SupplementaryClassesAndGroups,NumberofStudentsInOO,TheContingentOfPreschoolersStudyingOO,global_id,geoData
0,прочее,действующее,14500,государственное автономное учреждение города москвы медиацентр,ГАУ Медиацентр,NaN,7718924940,771801001,1137746210161,автономное,...,"[{'is_deleted': 0, 'Email': 'mc@edu.mos.ru', 'global_id': 106289}]",shkolamoskva.ru,,[],"[{'is_deleted': 0, 'LicenseAvailability': 'да', 'LicenseSeries': '', 'LicenseNumber': 'Л035-01298-77/00184946', 'LicenseExpires': 'бессрочная', 'AccreditationAvailability': 'нет', 'AccreditationSeries': '', 'AccreditationNumber': '', 'global_id': 16723, 'AccreditationExpires': ''}]",[],[],[],1139421628,"{'coordinates': [[[37.704597369, 55.80046703], [37.705485479, 55.800596013], [37.705559984, 55.800425152], [37.70563151, 55.800433527], [37.70589377, 55.79984388], [37.705771581, 55.799827129], [37.706009999, 55.799250874], [37.705658332, 55.79921067], [37.705402032, 55.799820428], [37.705554024, 55.79984053], [37.705524222, 55.799912561], [37.705136792, 55.799863982], [37.705014602, 55.800126979], [37.705413953, 55.800190634], [37.705327526, 55.800371548], [37.704963467, 55.800319428], [37.704975388, 55.800284809], [37.704886452, 55.80027104], [37.70486857, 55.800309568], [37.704677836, 55.800287791], [37.704597369, 55.80046703]]], 'type': 'Polygon'}"
1,организация дополнительного профессионального образования,действующее,14494,государственное автономное образовательное учреждение дополнительного профессионального образования города москвы корпоративный университет московского образования,ГАОУ ДПО «Корпоративный университет»,NaN,7714239823,771301001,1027700566761,автономное,...,"[{'is_deleted': 0, 'Email': 'cu@edu.mos.ru', 'global_id': 110905}]",corp-univer.ru,"Профессиональное образование | высшее образование - бакалавриат # Профессиональное образование | высшее образование - специалитет, магистратура # Профессиональное образование | высшее образование - подготовка кадров высшей квалификации # Дополнительное образование | дополнительное образование детей # Дополнительное образование | дополнительное профессиональное образование # Дополнительное образование | дополнительное образование взрослых",[],"[{'is_deleted': 0, 'LicenseAvailability': 'да', 'LicenseSeries': '', 'LicenseNumber': 'Л035-01298-77/00181973', 'LicenseExpires': 'бессрочная', 'AccreditationAvailability': 'нет', 'AccreditationSeries': '', 'AccreditationNumber': '', 'global_id': 21444, 'AccreditationExpires': ''}]",[],[],[],2386776971,"{'coordinates': [[[[37.539544671, 55.799821354], [37.53935791, 55.799897294], [37.53935791, 55.799975467], [37.539777129, 55.800289276], [37.539975812, 55.800201052], [37.539979785, 55.800139631], [37.539850642, 55.800040239], [37.539940049, 55.799996686], [37.539793024, 55.799887243], [37.539693683, 55.79993303], [37.539544671, 55.799821354]]], [[[37.560320716, 55.822874392], [37.560333905, 55.822858568], [37.560420206, 55.822748478], [37.56042418, 55.822673698], [37.560104302, 55.822584409], [37.560072512, 55.822634634], [37.560068539, 55.822653608], [37.560112249, 55.822669234], [37.559803149, 55.823045713], [37.559746525, 55.823031762], [37.559696854, 55.823077522], [37.559698841, 55.82309538], [37.559836925, 55.823132211], [37.559843879, 55.823189132], [37.560016732, 55.823232659], [37.560132961, 55.823104308], [37.560132961, 55.823047387], [37.560187599, 55.822975957], [37.560478668, 55.823051294], [37.560541253, 55.822973167], [37.560542247, 55.822953077], [37.560267072, 55.822881646], [37.56028098, 55.822865463], [37.560320716, 55.822874392]]], [[[37.618387714, 55.741878447], [37.618266121, 55.742030776], [37.618269697, 55.742087144], [37.618862167, 55.742235447], [37.618994489, 55.742067684], [37.618831173, 55.742023394], [37.618825212, 55.741984473], [37.618387714, 55.741878447]]], [[[37.557159683, 55

## Координаты из списка школ в результаты ЭГЕ

In [7]:
exams['geoData'] = exams['EDU_NAME'].apply(lambda x: schools.loc[schools['FullName'] == x, 'geoData'].values[0] if x in schools['FullName'].values else None)
exams.head()

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
0,8236,государственное бюджетное образовательное учреждение города москвы школа 390 имени генерала п.и. батова,797946035,2016-2017,Восточный административный округ,район Богородское,5,19,None
1,8237,государственное казенное образовательное учреждение города москвы кадетская школа-интернат 5 преображенский кадетский корпус,797949150,2016-2017,Восточный административный округ,район Богородское,14,47,"{'coordinates': [[[[37.710083127, 55.825085846], [37.709884644, 55.825088859], [37.70991683, 55.825557425], [37.710107267, 55.825552906], [37.710083127, 55.825085846]]], [[[37.710971466, 55.824666254], [37.710390321, 55.824675741], [37.710401249, 55.824826407], [37.710980407, 55.824816363], [37.710971466, 55.824666254]]], [[[37.710757032, 55.82506651], [37.710557953, 55.825070528], [37.710585371, 55.825566047], [37.710780874, 55.825561359], [37.710757032, 55.82506651]]], [[[37.707049481, 55.825154524], [37.707242202, 55.825149502], [37.707241209, 55.825030085], [37.70704054, 55.82503176], [37.707049481, 55.825154524]]], [[[37.710930347, 55.825726055], [37.711380243, 55.825718823], [37.711373091, 55.825557312], [37.711369514, 55.825539635], [37.710924625, 55.825550081], [37.710924625, 55.825570973], [37.710930347, 55.825726055]]], [[[37.583026114, 55.832647996], [37.582666499, 55.832611174], [37.582629743, 55.832706019], [37.582630736, 55.832758463], [37.58344732, 55.83284717], [37.583486063, 55.832741167], [37.583481096, 55.83270044], [37.583129428, 55.832659155], [37.583156251, 55.832582163], [37.584034426, 55.832677566], [37.584082109, 55.83255092], [37.584084096, 55.832518561], [37.583018166, 55.832398051], [37.582968496, 55.832531393], [37.582976443, 55.832562078], [37.583054923, 55.832573236], [37.583026114, 55.832647996]]], [[[37.709097862, 55.825586152], [37.70911789, 55.825928458], [37.70928669, 55.825930065], [37.709272385, 55.825774179], [37.70981884, 55.825766144], [37.709795952, 55.825566867], [37.709097862, 55.825586152]]]], 'type': 'MultiPolygon'}"
2,8238,государственное казенное образовательное учреждение города москвы специальная (коррекционная) образовательная школа-интернат 31,797949275,2016-2017,Восточный административный округ,район Вешняки,0,1,"{'coordinates': [[[[37.807984129, 55.726758653], [37.808307186, 55.726841896], [37.808526531, 55.726567328], [37.808196321, 55.726480727], [37.807984129, 55.726735157], [37.807984129, 55.726758653]]], [[[37.808976173, 55.726681066], [37.808727265, 55.726995241], [37.808704376, 55.727030686], [37.808858872, 55.727067742], [37.808921814, 55.726979129], [37.808944702, 55.72695174], [37.80899334, 55.726892127], [37.809216499, 55.726948517], [37.80932808, 55.726974296], [37.809522629, 55.72702263], [37.809602738, 55.726898572], [37.809597015, 55.72684057], [37.808976173, 55.726681066]]], [[[37.809706703, 55.726866063], [37.809706703, 55.726887545], [37.809673324, 55.72692044], [37.809673324, 55.726957362], [37.809852138, 55.727005025], [37.810194269, 55.726581425], [37.810197845, 55.726525035], [37.810020223, 55.7264767], [37.809706703, 55.726866063]]], [[[37.808979526, 55.726210184], [37.808632627, 55.726643858], [37.808636203, 55.726692864], [37.808812633, 55.726735828], [37.809155956, 55.726312897], [37.809153572, 55.726253149], [37.808979526, 55.726210184]]]], 'type': 'MultiPolygon'}"
3,8239,частное учреждение образовательная организация московская интернациональная школа планета,797949425,2016-2017,Восточный административный округ,район Вешняки,0,6,None
4,8240,автономная некоммерческая организация средняя образовательная школа колибри,797949570,2016-2017,Восточный административный округ,район Восточное Измайлово,1,5,None


In [8]:
exams['geoData'] = exams.apply(yandex_org, axis=1)
save_cache()

Загружено 31343 записей из cache.gz в кэш
Сохранено 31343 записей в cache.gz


In [9]:
exams[exams['geoData'].isna()]

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
573,8810,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,798000274,2016-2017,Юго-Восточный административный округ,Нижегородский район,5,22,None
1194,9432,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,880632637,2017-2018,Юго-Восточный административный округ,Нижегородский район,4,21,None
1524,9760,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,1011905093,2018-2019,Юго-Восточный административный округ,Нижегородский район,8,31,None


In [10]:
# Ручное добавление одной школы"
exams.loc[exams['EDU_NAME'] == "государственное бюджетное образовательное учреждение города москвы школа 439 интеллект", 'geoData'] = "{'type': 'Point', 'coordinates': [37.745376, 55.731212]}"
exams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5542 entries, 0 to 5541
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                5542 non-null   int64 
 1   EDU_NAME          5542 non-null   object
 2   global_id         5542 non-null   int64 
 3   YEAR              5542 non-null   object
 4   AdmArea           5542 non-null   object
 5   District          5542 non-null   object
 6   PASSES_OVER_220   5542 non-null   int64 
 7   PASSER_UNDER_160  5542 non-null   int64 
 8   geoData           5542 non-null   object
dtypes: int64(4), object(5)
memory usage: 389.8+ KB


In [11]:
save_dataset('exams')

Сохранено 5542 записей в exams.gz
